In [ ]:
!pip install transformers
!pip install sentencepiece
!pip install sacremoses

In [1]:
import pandas as pd
from tqdm import tqdm
from transformers import MarianMTModel, MarianTokenizer

model_name = '  '
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

def translate_text(text, model, tokenizer):
    inputs = tokenizer.encode(text, return_tensors="pt", padding=True, truncation=True)
    translated = model.generate(inputs, max_length=512)
    return tokenizer.decode(translated[0], skip_special_tokens=True)

In [6]:
translate_text('@user @user Btw. Are we now allowed to say \shithole\ countries? Are we also allowed to say how wes\u2026 @URL', model, tokenizer)

'@NNy_m. کیا اب ہم مُقدس ممالک کا کہنا ماننے کی اجازت دے رہے ہیں؟ کیا ہم یہ کہنے کی اجازت دے رہے ہیں کہ ہم کیسے ہیں'

In [7]:
translate_text('it truly fucking BAFFLES me when people genuinely fucking think that saying \u201cching chong ling lang ding\u201d is actuall\u2026 @URL', model, tokenizer)

'یہ واقعی غیر یقینی طور پر مجھے قتل کر رہا ہے جب لوگ سچ میں یہ کہتے ہیں کہ "وہ" ہے'

In [8]:
translate_text('This is the sort of thing that happens in a shithole country. @URL', model, tokenizer)

'یہ وہ چیز ہے جو ایک گندگی ملک میں واقع ہو رہی ہے۔'

In [ ]:
df = pd.read_csv(r'G:\My Drive\Hate Speech_Multilingual\Code\Dataset Statistics\english_curated(multi).csv')

total_rows = len(df)
with tqdm(total=total_rows, desc="Translating") as pbar:
    for index, row in df.iterrows():
        text = row['text']
        translated_text = translate_text(text, model, tokenizer)
        df.at[index, 'translated_text'] = translated_text   
        pbar.update(1) 

In [12]:
df['translated_text']

0         اگر امریکہ کے ایک اور 8 سال کی ہے تو ہم ایک گن...
1         @s_un soo انتہائی پریشان ہے ہر کوئی ایسا محسوس...
2         @وہنراساسسس نے ایک تصویر روشن کرنے والے بچے کو...
3         @Ny_m: کیا اب ہم مُقدس ممالک کا کہنا ماننے کی ...
4         @Iy'S اب بھی ایک تلخ ہمت ہوئی. اتنی زیادہ دلچس...
                                ...                        
131644                                                  nan
131645                                                  nan
131646                                                  nan
131647                                                  nan
131648                                                  nan
Name: translated_text, Length: 131649, dtype: object

In [17]:
len(df[df['translated_text'] == 'nan'])

101136

In [22]:
new_df = df[df['translated_text'] == 'nan']

#### NOTE: Due to sequentially translation on one node, I just break this one big file into 3 sub file inorder to use multiple nodes

In [27]:
part0 = new_df[:30000]
part1 = new_df[30000:62000]
part2 = new_df[62000:]

In [24]:
len(part1), len(part2)

(32000, 39136)

In [26]:
part1.to_csv("part1.csv")
part2.to_csv("part2.csv")

In [28]:
total_rows = len(part0)
with tqdm(total=total_rows, desc="Translating") as pbar:
    for index, row in part0.iterrows():
        text = row['text']
        translated_text = translate_text(text, model, tokenizer)
        part0.at[index, 'translated_text'] = translated_text   
        pbar.update(1) 

Translating: 100%|██████████| 30000/30000 [9:33:45<00:00,  1.15s/it]   


In [29]:
part0.head()

text  class  Disability  \
30513  The I in my name stand for, I will kill YOU ht...      0           0   
30514  Mudafucker, I'm writing you now to reply me or...      1           0   
30515  @POTUS Scholz in Biden and Nuland's little bit...      1           0   
30516  I ain’t got time for Nigger Knockers https://t...      1           0   
30517  @DrGuru_ When was Giannis a Center and his num...      0           0   

       Gender/Sexual  Origin/Nationality  Race/Ethnicity  Religion  \
30513              0                   0               0         0   
30514              0                   0               0         1   
30515              1                   0               0         0   
30516              0                   0               1         0   
30517              0                   0               0         0   

                                         translated_text  
30513  میں اپنے نام کے لئے کھڑے ہوں، میں آپ کو قتل کر...  
30514  مَیں آپ کو یہ جواب دینے کے لئے لکھ رہا ہوں کہ ...  
30515  @OOOOss اور مالا کی چھوٹی سی کتیا کو بچہ ہونے ...  
30516                      میں نے ابھی وقت نہیں ملا تھا.  
30517  @ReallyVirtual_بُس وقت ایک سینٹر تھا اور اس کی...

In [30]:
part0.to_csv("part0.csv")

In [31]:
df.head()

text  class  Disability  \
0  If America had another 8 years of Obama's ideo...      1           0   
1  @user queue is fucking retarded it makes every...      1           1   
2  @user Que ce ne soit pas des Burundais refugie...      1           0   
3  @user @user Btw. Are we now allowed to say \sh...      0           0   
4  @user Still a bitter cunt. Why so much interes...      1           0   

   Gender/Sexual  Origin/Nationality  Race/Ethnicity  Religion  \
0              0                   1               0         0   
1              0                   0               0         0   
2              0                   1               0         0   
3              0                   0               0         0   
4              1                   0               0         0   

                                     translated_text  
0  اگر امریکہ کے ایک اور 8 سال کی ہے تو ہم ایک گن...  
1  @s_un soo انتہائی پریشان ہے ہر کوئی ایسا محسوس...  
2  @وہنراساسسس نے ایک تصویر روشن کرنے والے بچے کو...  
3  @Ny_m: کیا اب ہم مُقدس ممالک کا کہنا ماننے کی ...  
4  @Iy'S اب بھی ایک تلخ ہمت ہوئی. اتنی زیادہ دلچس...

In [32]:
df.to_csv("english_curated(multi)_translated.csv")

### merging all parts of dataset from different systems 

After, the translation completed over multiple nodes, all files merge together back.

In [45]:
import pandas as pd

df1 = df[df['translated_text'] != 'nan']
df2 = pd.read_csv('part0.csv')
df3 = pd.read_csv('part1(translated).csv')
df4 = pd.read_csv('part2(translated).csv')

In [46]:
len(df1), len(df2), len(df3), len(df4)

(30513, 30000, 32000, 39136)

In [ ]:
merged_df = pd.concat([df1, df2, df3, df4], ignore_index=True)

In [49]:
merged_df.columns

Index(['text', 'class', 'Disability', 'Gender/Sexual', 'Origin/Nationality',
       'Race/Ethnicity', 'Religion', 'translated_text', 'Unnamed: 0',
       'Unnamed: 0.1'],
      dtype='object')

In [ ]:
merged_df = merged_df[['text',  'translated_text', 'class', 'Disability', 'Gender/Sexual', 'Origin/Nationality',
       'Race/Ethnicity', 'Religion'] ]

merged_df.columns = ['english_text',  'urdu_text', 'class', 'Disability', 'Gender/Sexual', 'Origin/Nationality',
       'Race/Ethnicity', 'Religion']

In [50]:
merged_df.to_csv('english_urdu_curated_and_translated.csv', index=False)

In [51]:
old  = pd.read_csv('english_curated(multi).csv')
new = pd.read_csv('english_urdu_curated_and_translated.csv')

len(old), len(new)

(131649, 131649)

In [52]:
old.head()

text  class  Disability  \
0  If America had another 8 years of Obama's ideo...      1           0   
1  @user queue is fucking retarded it makes every...      1           1   
2  @user Que ce ne soit pas des Burundais refugie...      1           0   
3  @user @user Btw. Are we now allowed to say \sh...      0           0   
4  @user Still a bitter cunt. Why so much interes...      1           0   

   Gender/Sexual  Origin/Nationality  Race/Ethnicity  Religion  
0              0                   1               0         0  
1              0                   0               0         0  
2              0                   1               0         0  
3              0                   0               0         0  
4              1                   0               0         0

In [53]:
new.head()

english_text  \
0  If America had another 8 years of Obama's ideo...   
1  @user queue is fucking retarded it makes every...   
2  @user Que ce ne soit pas des Burundais refugie...   
3  @user @user Btw. Are we now allowed to say \sh...   
4  @user Still a bitter cunt. Why so much interes...   

                                           urdu_text  class  Disability  \
0  اگر امریکہ کے ایک اور 8 سال کی ہے تو ہم ایک گن...      1           0   
1  @s_un soo انتہائی پریشان ہے ہر کوئی ایسا محسوس...      1           1   
2  @وہنراساسسس نے ایک تصویر روشن کرنے والے بچے کو...      1           0   
3  @Ny_m: کیا اب ہم مُقدس ممالک کا کہنا ماننے کی ...      0           0   
4  @Iy'S اب بھی ایک تلخ ہمت ہوئی. اتنی زیادہ دلچس...      1           0   

   Gender/Sexual  Origin/Nationality  Race/Ethnicity  Religion  
0              0                   1               0         0  
1              0                   0               0         0  
2              0                   1               0         0  
3              0                   0               0         0  
4              1                   0               0         0